In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier 
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, KFold, LeaveOneOut
from sklearn.preprocessing import Imputer
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, auc, roc_auc_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from random import randint
import matplotlib.pyplot as plt
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
import category_encoders as ce
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from catboost import CatBoostClassifier

In [2]:
train_A = pd.read_csv('data/A_hhold_train.csv', index_col='id')
train_B = pd.read_csv('data/B_hhold_train.csv', index_col='id')
train_C = pd.read_csv('data/C_hhold_train.csv', index_col='id')

In [3]:
test_A = pd.read_csv('data/A_hhold_test.csv', index_col='id')
test_B = pd.read_csv('data/B_hhold_test.csv', index_col='id')
test_C = pd.read_csv('data/C_hhold_test.csv', index_col='id')

In [4]:
def make_country_sub(preds, test_feat, country):
    # make sure we code the country correctly
    country_codes = ['A', 'B', 'C']
    
    # get just the poor probabilities
    country_sub = pd.DataFrame(data=preds[:, 1],  # proba p=1
                               columns=['poor'], 
                               index=test_feat.index)

    
    # add the country code for joining later
    country_sub["country"] = country
    return country_sub[["country", "poor"]]

In [5]:
#funcetion used to centrailize the numeric data 
def standardize(df, numeric_only=True): 
    
    """
    :type df: pd.DataFrame
    :type numeric_only: bool
    
    :rtype df: pd.DataFrame
    """
    
    numeric = df.select_dtypes(include=['int64', 'float64'])
    
    # subtracy mean and divide by std
    df[numeric.columns] = (numeric - numeric.mean()) / numeric.std()
    
    return df

In [6]:
#function used to preprocess the data(simply drop the poor and country columns)
def preporcess(original_train, original_test, op='one_hot'):
    
    """
    :type original_train: pd.DataFrame
    :type original_test: pd.DataFrame
    :type op: string
    
    :rtype x_train: pd.DataFrame
    :rtype y_train: pd.DataFrame
    :rtype x_test: pd.DataFrame
    """
    
    #Get the copy of orginal data
    x_train = original_train.copy()
    x_test = original_test.copy()

    #Get the label of the train set
    y_train = pd.DataFrame(x_train['poor'])
    
    #drop the 'poor' and 'country' columns
    x_train = x_train.drop(['poor', 'country'], axis=1)
    x_test = x_test.drop(['country'], axis=1)
    
    #get the list of coulumns that need to be encoded
    columns = list(x_train.select_dtypes(include=['object']).columns)
    
    #encode the features according to the 'op' operations
    if op == 'one_hot':
        encoder = ce.one_hot.OneHotEncoder(cols=columns)
    elif op == 'binary':
        encoder = ce.binary.BinaryEncoder(cols=columns)
    elif op == 'ordinal':
        encoder = ce.ordinal.OrdinalEncoder(cols=columns)
    elif op == 'backward':
        encoder = ce.backward_difference.BackwardDifferenceEncoder(cols=columns)
    elif op == 'polynomial':
        encoder = ce.polynomial.PolynomialEncoder(cols=columns)
    elif op == 'sum':
        encoder = ce.sum_coding.SumEncoder(cols=columns)
    elif op == 'hashing':
        encoder = ce.hashing.HashingEncoder(cols=columns)
    elif op == 'helmert':
        encoder = ce.helmert.HelmertEncoder(cols=columns)
    
    #fit the encoder and transform the datasets
    encoder.fit(x_train)
    x_train = encoder.transform(x_train)
    x_test = encoder.transform(x_test)
    
    #for columns with Nan, plug in with median value
    imputed_train = x_train.copy()
    imputed_test = x_test.copy()
    
    # make new columns indicating what will be imputed
    cols_with_missing = (col for col in x_train.columns 
                                 if x_train[col].isnull().any())
    
    #insert new colums and tell which row is missing 
    for col in cols_with_missing:
        imputed_train[col + '_was_missing'] = imputed_train[col].isnull()
        imputed_test[col + '_was_missing'] = imputed_test[col].isnull()
    
    imp = Imputer(missing_values='NaN', strategy='median',axis=0)
    x_train = imp.fit_transform(imputed_train)
    x_test = imp.transform(imputed_test)
       
    return x_train, y_train.values.ravel(), x_test

In [7]:
x_train_A, y_train_A, x_test_A = preporcess(original_train=train_A, original_test=test_A, op='one_hot')
x_train_B, y_train_B, x_test_B = preporcess(original_train=train_B, original_test=test_B, op='one_hot')
x_train_C, y_train_C, x_test_C = preporcess(original_train=train_C, original_test=test_C, op='one_hot')

In [31]:
def cross_validate(model, X, y, cv=5, oversample=False, early_stop=False):
    
    seed = randint(1,9999)
    result_log_loss = np.zeros(cv)
    result_roc_auc = np.zeros(cv)
    
    kf = StratifiedKFold(n_splits=cv)
    
    i = 0
    
    for train_index, dev_index in kf.split(X,y):
        x_train = X[train_index]
        y_train = y[train_index]
        x_dev = X[dev_index]
        y_dev = y[dev_index]
        
        if oversample == True:
            x_train, y_train = RandomOverSampler().fit_sample(x_train, y_train)
            
        if early_stop == True:
            model.fit(X=x_train, y=y_train, eval_set=[x_dev,y_dev])
#             model.fit(X=x_train, y=y_train, eval_set=[(x_train,y_train),(x_dev,y_dev)], eval_metric='logloss', early_stopping_rounds=20)
        else:
            model.fit(x_train, y_train)
        
        pred_proba = model.predict_proba(x_dev)[:,-1]
        
        result_log_loss[i] = log_loss(y_pred=pred_proba, y_true=y_dev)
        result_roc_auc[i] = roc_auc_score(y_score=pred_proba, y_true=y_dev)
        
        i += 1
        
    return result_log_loss.mean(), result_roc_auc.mean()

In [9]:
def gridSearch(model, grid_params, X, y, oversample=False, early_stop=False, cv=5):
    
    grids = ParameterGrid(grid_params)
    result_score = np.zeros([len(grids), 2])
    
    i = 0
    
    for grid in grids:
        model.set_params(**grid)
        logScore, aucScore = cross_validate(model, X, y, cv, oversample, early_stop)
        result_score[i][0] = logScore
        result_score[i][1] = aucScore
        i += 1
    
    return grids, result_score

In [10]:
def model_feature_evaluate(feature_model, predict_model, X, y, oversample=False, early_stop=False, cv=5):
    feature_model.fit(X,y)
    feature_model = SelectFromModel(feature_model, prefit=True)
    X_new = feature_model.transform(X)
    logScore, aucScore = cross_validate(predict_model, X_new, y, cv, oversample, early_stop)
    return logScore, aucScore

def base_model_evaluate(model, X, y, oversample=False, early_stop=False, cv=5):
    logScore, aucScore = cross_validate(model, X, y, cv, oversample, early_stop)
    return logScore, aucScore

def feature_transform(feature_model, X_train, Y_train, X_test):
    feature_model.fit(X_train, Y_train)
    model = SelectFromModel(feature_model, prefit=True)
    X_train_new = model.transform(X_train)
    X_test_new = model.transform(X_test)
    return X_train_new, X_test_new

In [11]:
x_train_new_A, x_test_new_A = feature_transform(XGBClassifier(), x_train_A, y_train_A, x_test_A)
x_train_new_B, x_test_new_B = feature_transform(XGBClassifier(), x_train_B, y_train_B, x_test_B)
x_train_new_C, x_test_new_C = feature_transform(XGBClassifier(), x_train_C, y_train_C, x_test_C)

# Country A Model Parameters

In [157]:
#xgb data
#lr
lr_params_A = {}
lr_params_A['C'] = 1
lr_params_A['penalty'] = 'l1'

#xgb
xgb_params_A = {}
xgb_params_A['learning_rate'] = 0.05
xgb_params_A['n_estimators'] = 800
xgb_params_A['max_depth'] = 3
xgb_params_A['colsample_bytree'] = 0.9


#lgb
lgb_params_A = {}
lgb_params_A['learning_rate'] = 0.05
lgb_params_A['num_iterations'] = 800
lgb_params_A['max_depth'] = 3
lgb_params_A['colsample_bytree'] = 0.6
lgb_params_A['min_child_weight'] = 4

#cat
cat_params_A = {}
cat_params_A['learning_rate'] = 0.05
cat_params_A['iterations'] = 750
cat_params_A['depth'] = 5
cat_params_A['loss_function'] = 'Logloss'

# Country B Model Parameters

In [155]:
#xgb data
#lr
lr_params_B = {}
lr_params_B['C'] = 0.5
lr_params_B['penalty'] = 'l2'

#lgb
lgb_params_B = {}
lgb_params_B['learning_rate'] = 0.01
lgb_params_B['max_depth'] = 8
lgb_params_B['num_iterations'] = 1000
lgb_params_B['colsample_bytree'] = 0.4
lgb_params_B['subsample'] = 0.7

#xgb
xgb_params_B = {}
xgb_params_B['learning_rate'] = 0.05
xgb_params_B['n_estimators'] = 500
xgb_params_B['min_child_weight'] = 2
xgb_params_B['colsample_bytree'] = 0.8
xgb_params_B['max_delta_step'] = 3

#cat
cat_params_B = {}
cat_params_B['learning_rate'] = 0.03
cat_params_B['iterations'] = 700
cat_params_B['depth'] = 3
cat_params_B['bagging_temperature'] = 0.8
cat_params_B['l2_leaf_reg'] = 7
cat_params_B['loss_function'] = 'Logloss'

# Country C Model Parameters

In [156]:
#xgb data
#lr
lr_params_C = {}
lr_params_C['C'] = 1
lr_params_C['penalty'] = 'l1'

#xgb
xgb_params_C = {}
xgb_params_C['learning_rate'] = 0.05
xgb_params_C['n_estimators'] = 300
xgb_params_C['max_depth'] = 3
xgb_params_C['colsample_bytree'] = 0.6
xgb_params_C['max_delta_step'] = 3

#lgb
lgb_params_C = {}
lgb_params_C['learning_rate'] = 0.05
lgb_params_C['num_iterations'] = 150
lgb_params_C['max_depth'] = 4

#cat
cat_params_C = {}
cat_params_C['learning_rate'] = 0.05
cat_params_C['iterations'] = 700
cat_params_C['depth'] = 3
cat_params_C['bagging_temperature'] = 1
cat_params_C['l2_leaf_reg'] = 5
cat_params_C['loss_function'] = 'Logloss'

# Parameters tunning

In [165]:
params = {}
params['weights'] = [[1,1,1,1], [3,1,1,1], [1,3,1,1], [1,1,3,1], [1,1,1,3]]

In [169]:
p, s = gridSearch(model_C, params, x_train_new_C, y_train_C)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.5775390	total: 29.5ms	remaining: 20.6s
1:	learn: 0.4833360	total: 71ms	remaining: 24.8s
2:	learn: 0.4101444	total: 112ms	remaining: 26.1s
3:	learn: 0.3462082	total: 138ms	remaining: 24s
4:	learn: 0.2966534	total: 167ms	remaining: 23.3s
5:	learn: 0.2531204	total: 178ms	remaining: 20.6s
6:	learn: 0.2236687	total: 188ms	remaining: 18.6s
7:	learn: 0.1937398	total: 197ms	remaining: 17.1s
8:	learn: 0.1698703	total: 207ms	remaining: 15.9s
9:	learn: 0.1506136	total: 218ms	remaining: 15s
10:	learn: 0.1330966	total: 227ms	remaining: 14.2s
11:	learn: 0.1158670	total: 253ms	remaining: 14.5s
12:	learn: 0.1046665	total: 279ms	remaining: 14.7s
13:	learn: 0.0958141	total: 308ms	remaining: 15.1s
14:	learn: 0.0827119	total: 324ms	remaining: 14.8s
15:	learn: 0.0760934	total: 336ms	remaining: 14.4s
16:	learn: 0.0708812	total: 348ms	remaining: 14s
17:	learn: 0.0619384	total: 357ms	remaining: 13.5s
18:	learn: 0.0587690	total: 367ms	remaining: 13.1s
19:	learn: 0.0559740	total: 376ms	remaining: 12

163:	learn: 0.0119844	total: 2.16s	remaining: 7.05s
164:	learn: 0.0119552	total: 2.17s	remaining: 7.05s
165:	learn: 0.0119158	total: 2.19s	remaining: 7.04s
166:	learn: 0.0118957	total: 2.2s	remaining: 7.01s
167:	learn: 0.0118539	total: 2.21s	remaining: 6.99s
168:	learn: 0.0118144	total: 2.22s	remaining: 6.97s
169:	learn: 0.0117937	total: 2.23s	remaining: 6.96s
170:	learn: 0.0117716	total: 2.24s	remaining: 6.94s
171:	learn: 0.0117143	total: 2.25s	remaining: 6.92s
172:	learn: 0.0116708	total: 2.26s	remaining: 6.9s
173:	learn: 0.0116245	total: 2.27s	remaining: 6.87s
174:	learn: 0.0115493	total: 2.28s	remaining: 6.85s
175:	learn: 0.0115071	total: 2.29s	remaining: 6.83s
176:	learn: 0.0114620	total: 2.3s	remaining: 6.81s
177:	learn: 0.0114427	total: 2.31s	remaining: 6.79s
178:	learn: 0.0114023	total: 2.32s	remaining: 6.77s
179:	learn: 0.0113633	total: 2.33s	remaining: 6.74s
180:	learn: 0.0113263	total: 2.34s	remaining: 6.72s
181:	learn: 0.0112783	total: 2.35s	remaining: 6.7s
182:	learn: 0.01

322:	learn: 0.0070943	total: 3.88s	remaining: 4.53s
323:	learn: 0.0070873	total: 3.9s	remaining: 4.52s
324:	learn: 0.0070728	total: 3.91s	remaining: 4.51s
325:	learn: 0.0070539	total: 3.92s	remaining: 4.5s
326:	learn: 0.0070329	total: 3.93s	remaining: 4.48s
327:	learn: 0.0070132	total: 3.94s	remaining: 4.47s
328:	learn: 0.0070062	total: 3.95s	remaining: 4.45s
329:	learn: 0.0069983	total: 3.96s	remaining: 4.44s
330:	learn: 0.0069841	total: 3.97s	remaining: 4.42s
331:	learn: 0.0069762	total: 3.98s	remaining: 4.41s
332:	learn: 0.0069614	total: 3.99s	remaining: 4.39s
333:	learn: 0.0069349	total: 4s	remaining: 4.38s
334:	learn: 0.0069327	total: 4s	remaining: 4.36s
335:	learn: 0.0069004	total: 4.01s	remaining: 4.35s
336:	learn: 0.0068873	total: 4.02s	remaining: 4.33s
337:	learn: 0.0068783	total: 4.03s	remaining: 4.32s
338:	learn: 0.0068476	total: 4.04s	remaining: 4.31s
339:	learn: 0.0068402	total: 4.05s	remaining: 4.29s
340:	learn: 0.0068242	total: 4.06s	remaining: 4.28s
341:	learn: 0.006802

485:	learn: 0.0051540	total: 5.62s	remaining: 2.47s
486:	learn: 0.0051410	total: 5.64s	remaining: 2.47s
487:	learn: 0.0051326	total: 5.65s	remaining: 2.46s
488:	learn: 0.0051307	total: 5.67s	remaining: 2.44s
489:	learn: 0.0051186	total: 5.67s	remaining: 2.43s
490:	learn: 0.0051099	total: 5.68s	remaining: 2.42s
491:	learn: 0.0051094	total: 5.69s	remaining: 2.41s
492:	learn: 0.0051054	total: 5.7s	remaining: 2.4s
493:	learn: 0.0050910	total: 5.71s	remaining: 2.38s
494:	learn: 0.0050819	total: 5.72s	remaining: 2.37s
495:	learn: 0.0050759	total: 5.73s	remaining: 2.36s
496:	learn: 0.0050638	total: 5.74s	remaining: 2.35s
497:	learn: 0.0050586	total: 5.75s	remaining: 2.33s
498:	learn: 0.0050390	total: 5.76s	remaining: 2.32s
499:	learn: 0.0050343	total: 5.77s	remaining: 2.31s
500:	learn: 0.0050304	total: 5.78s	remaining: 2.3s
501:	learn: 0.0050228	total: 5.79s	remaining: 2.28s
502:	learn: 0.0050202	total: 5.8s	remaining: 2.27s
503:	learn: 0.0050119	total: 5.81s	remaining: 2.26s
504:	learn: 0.00

...

169:	learn: 0.0109473	total: 1.96s	remaining: 6.11s
170:	learn: 0.0109085	total: 1.98s	remaining: 6.13s
171:	learn: 0.0108479	total: 1.99s	remaining: 6.12s
172:	learn: 0.0107861	total: 2s	remaining: 6.1s
173:	learn: 0.0107491	total: 2.01s	remaining: 6.09s
174:	learn: 0.0107127	total: 2.02s	remaining: 6.07s
175:	learn: 0.0106871	total: 2.03s	remaining: 6.05s
176:	learn: 0.0106561	total: 2.04s	remaining: 6.04s
177:	learn: 0.0106227	total: 2.05s	remaining: 6.02s
178:	learn: 0.0105906	total: 2.06s	remaining: 6s
179:	learn: 0.0105733	total: 2.07s	remaining: 5.99s
180:	learn: 0.0105247	total: 2.08s	remaining: 5.97s
181:	learn: 0.0104922	total: 2.09s	remaining: 5.95s
182:	learn: 0.0104654	total: 2.1s	remaining: 5.94s
183:	learn: 0.0104325	total: 2.11s	remaining: 5.92s
184:	learn: 0.0103799	total: 2.12s	remaining: 5.91s
185:	learn: 0.0103364	total: 2.13s	remaining: 5.89s
186:	learn: 0.0103154	total: 2.14s	remaining: 5.88s
187:	learn: 0.0102852	total: 2.15s	remaining: 5.86s
188:	learn: 0.010237

344:	learn: 0.0065890	total: 3.87s	remaining: 3.98s
345:	learn: 0.0065873	total: 3.89s	remaining: 3.98s
346:	learn: 0.0065501	total: 3.9s	remaining: 3.96s
347:	learn: 0.0065354	total: 3.91s	remaining: 3.95s
348:	learn: 0.0065178	total: 3.92s	remaining: 3.94s
349:	learn: 0.0064906	total: 3.93s	remaining: 3.93s
350:	learn: 0.0064612	total: 3.94s	remaining: 3.91s
351:	learn: 0.0064510	total: 3.95s	remaining: 3.9s
352:	learn: 0.0064335	total: 3.96s	remaining: 3.89s
353:	learn: 0.0064245	total: 3.97s	remaining: 3.88s
354:	learn: 0.0064073	total: 3.98s	remaining: 3.87s
355:	learn: 0.0063995	total: 3.99s	remaining: 3.85s
356:	learn: 0.0063619	total: 4s	remaining: 3.84s
357:	learn: 0.0063578	total: 4.01s	remaining: 3.83s
358:	learn: 0.0063489	total: 4.02s	remaining: 3.81s
359:	learn: 0.0063275	total: 4.03s	remaining: 3.8s
360:	learn: 0.0063130	total: 4.04s	remaining: 3.79s
361:	learn: 0.0062985	total: 4.04s	remaining: 3.78s
362:	learn: 0.0062727	total: 4.05s	remaining: 3.76s
363:	learn: 0.0062

520:	learn: 0.0043567	total: 5.8s	remaining: 1.99s
521:	learn: 0.0043484	total: 5.82s	remaining: 1.98s
522:	learn: 0.0043459	total: 5.83s	remaining: 1.97s
523:	learn: 0.0043281	total: 5.84s	remaining: 1.96s
524:	learn: 0.0043268	total: 5.85s	remaining: 1.95s
525:	learn: 0.0043221	total: 5.86s	remaining: 1.94s
526:	learn: 0.0043168	total: 5.87s	remaining: 1.93s
527:	learn: 0.0043118	total: 5.87s	remaining: 1.91s
528:	learn: 0.0043024	total: 5.88s	remaining: 1.9s
529:	learn: 0.0042936	total: 5.89s	remaining: 1.89s
530:	learn: 0.0042905	total: 5.91s	remaining: 1.88s
531:	learn: 0.0042879	total: 5.91s	remaining: 1.87s
532:	learn: 0.0042869	total: 5.92s	remaining: 1.86s
533:	learn: 0.0042619	total: 5.93s	remaining: 1.84s
534:	learn: 0.0042532	total: 5.94s	remaining: 1.83s
535:	learn: 0.0042420	total: 5.95s	remaining: 1.82s
536:	learn: 0.0042279	total: 5.96s	remaining: 1.81s
537:	learn: 0.0042258	total: 5.97s	remaining: 1.8s
538:	learn: 0.0042117	total: 5.98s	remaining: 1.79s
539:	learn: 0.0

679:	learn: 0.0033172	total: 7.53s	remaining: 221ms
680:	learn: 0.0033118	total: 7.55s	remaining: 211ms
681:	learn: 0.0033100	total: 7.56s	remaining: 200ms
682:	learn: 0.0033091	total: 7.57s	remaining: 188ms
683:	learn: 0.0033038	total: 7.58s	remaining: 177ms
684:	learn: 0.0032969	total: 7.59s	remaining: 166ms
685:	learn: 0.0032941	total: 7.6s	remaining: 155ms
686:	learn: 0.0032824	total: 7.61s	remaining: 144ms
687:	learn: 0.0032753	total: 7.62s	remaining: 133ms
688:	learn: 0.0032692	total: 7.63s	remaining: 122ms
689:	learn: 0.0032689	total: 7.64s	remaining: 111ms
690:	learn: 0.0032684	total: 7.65s	remaining: 99.6ms
691:	learn: 0.0032680	total: 7.66s	remaining: 88.6ms
692:	learn: 0.0032652	total: 7.67s	remaining: 77.5ms
693:	learn: 0.0032629	total: 7.68s	remaining: 66.4ms
694:	learn: 0.0032625	total: 7.69s	remaining: 55.3ms
695:	learn: 0.0032561	total: 7.7s	remaining: 44.2ms
696:	learn: 0.0032532	total: 7.71s	remaining: 33.2ms
697:	learn: 0.0032405	total: 7.72s	remaining: 22.1ms
698:	l

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [170]:
list(p)

[{'weights': [1, 1, 1, 1]},
 {'weights': [3, 1, 1, 1]},
 {'weights': [1, 3, 1, 1]},
 {'weights': [1, 1, 3, 1]},
 {'weights': [1, 1, 1, 3]}]

In [171]:
s

array([[0.01709116, 0.99957121],
       [0.01639818, 0.99957492],
       [0.01605069, 0.99958609],
       [0.01671343, 0.99955158],
       [0.02034652, 0.99951046]])

In [148]:
s

array([[0.02291786, 0.99868841],
       [0.0175257 , 0.99932498],
       [0.0173191 , 0.99937022],
       [0.01738017, 0.99936151],
       [0.01745757, 0.99930005],
       [0.01768471, 0.99932113],
       [0.01736526, 0.99942523]])

In [134]:
base_model_evaluate(CatBoostClassifier(learning_rate=0.05, iterations=800,use_best_model=True), x_train_new_C, y_train_C,early_stop=True)

0:	learn: 0.5856761	test: 0.5850599	best: 0.5850599 (0)	total: 17.9ms	remaining: 14.3s
1:	learn: 0.4899577	test: 0.4892904	best: 0.4892904 (1)	total: 51ms	remaining: 20.4s
2:	learn: 0.4084129	test: 0.4074015	best: 0.4074015 (2)	total: 67.3ms	remaining: 17.9s
3:	learn: 0.3460540	test: 0.3440830	best: 0.3440830 (3)	total: 85.4ms	remaining: 17s
4:	learn: 0.2959123	test: 0.2938772	best: 0.2938772 (4)	total: 112ms	remaining: 17.8s
5:	learn: 0.2530508	test: 0.2504387	best: 0.2504387 (5)	total: 129ms	remaining: 17.1s
6:	learn: 0.2183917	test: 0.2159245	best: 0.2159245 (6)	total: 158ms	remaining: 17.9s
7:	learn: 0.1908258	test: 0.1882827	best: 0.1882827 (7)	total: 175ms	remaining: 17.4s
8:	learn: 0.1663981	test: 0.1635652	best: 0.1635652 (8)	total: 205ms	remaining: 18s
9:	learn: 0.1467951	test: 0.1436372	best: 0.1436372 (9)	total: 225ms	remaining: 17.8s
10:	learn: 0.1299764	test: 0.1270194	best: 0.1270194 (10)	total: 258ms	remaining: 18.5s
11:	learn: 0.1169553	test: 0.1137188	best: 0.1137188 (

95:	learn: 0.0090521	test: 0.0143585	best: 0.0143585 (95)	total: 1.93s	remaining: 14.2s
96:	learn: 0.0089772	test: 0.0143270	best: 0.0143270 (96)	total: 1.95s	remaining: 14.1s
97:	learn: 0.0088687	test: 0.0142884	best: 0.0142884 (97)	total: 1.97s	remaining: 14.1s
98:	learn: 0.0088093	test: 0.0142950	best: 0.0142884 (97)	total: 1.99s	remaining: 14.1s
99:	learn: 0.0087572	test: 0.0143545	best: 0.0142884 (97)	total: 2s	remaining: 14s
100:	learn: 0.0086643	test: 0.0143881	best: 0.0142884 (97)	total: 2.02s	remaining: 14s
101:	learn: 0.0085798	test: 0.0144263	best: 0.0142884 (97)	total: 2.04s	remaining: 14s
102:	learn: 0.0084772	test: 0.0143691	best: 0.0142884 (97)	total: 2.06s	remaining: 13.9s
103:	learn: 0.0083272	test: 0.0143680	best: 0.0142884 (97)	total: 2.08s	remaining: 13.9s
104:	learn: 0.0082447	test: 0.0142967	best: 0.0142884 (97)	total: 2.1s	remaining: 13.9s
105:	learn: 0.0081971	test: 0.0143344	best: 0.0142884 (97)	total: 2.11s	remaining: 13.8s
106:	learn: 0.0081261	test: 0.014276

188:	learn: 0.0037862	test: 0.0133580	best: 0.0133580 (188)	total: 3.68s	remaining: 11.9s
189:	learn: 0.0037721	test: 0.0133482	best: 0.0133482 (189)	total: 3.7s	remaining: 11.9s
190:	learn: 0.0037393	test: 0.0133415	best: 0.0133415 (190)	total: 3.72s	remaining: 11.9s
191:	learn: 0.0037201	test: 0.0133751	best: 0.0133415 (190)	total: 3.74s	remaining: 11.8s
192:	learn: 0.0036965	test: 0.0133924	best: 0.0133415 (190)	total: 3.75s	remaining: 11.8s
193:	learn: 0.0036698	test: 0.0133132	best: 0.0133132 (193)	total: 3.77s	remaining: 11.8s
194:	learn: 0.0036517	test: 0.0132860	best: 0.0132860 (194)	total: 3.79s	remaining: 11.8s
195:	learn: 0.0035896	test: 0.0132771	best: 0.0132771 (195)	total: 3.81s	remaining: 11.7s
196:	learn: 0.0035658	test: 0.0132524	best: 0.0132524 (196)	total: 3.83s	remaining: 11.7s
197:	learn: 0.0035480	test: 0.0132058	best: 0.0132058 (197)	total: 3.84s	remaining: 11.7s
198:	learn: 0.0035191	test: 0.0132273	best: 0.0132058 (197)	total: 3.86s	remaining: 11.7s
199:	learn:

284:	learn: 0.0020087	test: 0.0132473	best: 0.0130810 (278)	total: 5.57s	remaining: 10.1s
285:	learn: 0.0020006	test: 0.0132742	best: 0.0130810 (278)	total: 5.58s	remaining: 10s
286:	learn: 0.0019960	test: 0.0132491	best: 0.0130810 (278)	total: 5.6s	remaining: 10s
287:	learn: 0.0019793	test: 0.0132560	best: 0.0130810 (278)	total: 5.62s	remaining: 9.99s
288:	learn: 0.0019732	test: 0.0132227	best: 0.0130810 (278)	total: 5.64s	remaining: 9.97s
289:	learn: 0.0019574	test: 0.0131866	best: 0.0130810 (278)	total: 5.65s	remaining: 9.94s
290:	learn: 0.0019520	test: 0.0132062	best: 0.0130810 (278)	total: 5.67s	remaining: 9.92s
291:	learn: 0.0019473	test: 0.0132080	best: 0.0130810 (278)	total: 5.69s	remaining: 9.9s
292:	learn: 0.0019222	test: 0.0132276	best: 0.0130810 (278)	total: 5.71s	remaining: 9.88s
293:	learn: 0.0019150	test: 0.0132191	best: 0.0130810 (278)	total: 5.74s	remaining: 9.88s
294:	learn: 0.0019052	test: 0.0132274	best: 0.0130810 (278)	total: 5.76s	remaining: 9.86s
295:	learn: 0.00

380:	learn: 0.0012470	test: 0.0133924	best: 0.0130810 (278)	total: 7.38s	remaining: 8.12s
381:	learn: 0.0012449	test: 0.0133820	best: 0.0130810 (278)	total: 7.4s	remaining: 8.1s
382:	learn: 0.0012378	test: 0.0133521	best: 0.0130810 (278)	total: 7.42s	remaining: 8.08s
383:	learn: 0.0012365	test: 0.0133491	best: 0.0130810 (278)	total: 7.43s	remaining: 8.05s
384:	learn: 0.0012278	test: 0.0133614	best: 0.0130810 (278)	total: 7.45s	remaining: 8.03s
385:	learn: 0.0012174	test: 0.0133648	best: 0.0130810 (278)	total: 7.47s	remaining: 8.01s
386:	learn: 0.0012133	test: 0.0134002	best: 0.0130810 (278)	total: 7.49s	remaining: 7.99s
387:	learn: 0.0012109	test: 0.0133937	best: 0.0130810 (278)	total: 7.51s	remaining: 7.97s
388:	learn: 0.0012082	test: 0.0133808	best: 0.0130810 (278)	total: 7.53s	remaining: 7.95s
389:	learn: 0.0012007	test: 0.0133480	best: 0.0130810 (278)	total: 7.54s	remaining: 7.93s
390:	learn: 0.0011977	test: 0.0133459	best: 0.0130810 (278)	total: 7.57s	remaining: 7.91s
391:	learn: 

...

443:	learn: 0.0011335	test: 0.0265322	best: 0.0227639 (158)	total: 8.26s	remaining: 6.63s
444:	learn: 0.0011308	test: 0.0265587	best: 0.0227639 (158)	total: 8.28s	remaining: 6.61s
445:	learn: 0.0011285	test: 0.0265925	best: 0.0227639 (158)	total: 8.3s	remaining: 6.59s
446:	learn: 0.0011238	test: 0.0266032	best: 0.0227639 (158)	total: 8.32s	remaining: 6.57s
447:	learn: 0.0011194	test: 0.0266042	best: 0.0227639 (158)	total: 8.33s	remaining: 6.55s
448:	learn: 0.0011178	test: 0.0266118	best: 0.0227639 (158)	total: 8.35s	remaining: 6.53s
449:	learn: 0.0011159	test: 0.0266287	best: 0.0227639 (158)	total: 8.37s	remaining: 6.51s
450:	learn: 0.0011112	test: 0.0266756	best: 0.0227639 (158)	total: 8.38s	remaining: 6.49s
451:	learn: 0.0011078	test: 0.0267300	best: 0.0227639 (158)	total: 8.4s	remaining: 6.47s
452:	learn: 0.0011059	test: 0.0267474	best: 0.0227639 (158)	total: 8.42s	remaining: 6.45s
453:	learn: 0.0011029	test: 0.0267898	best: 0.0227639 (158)	total: 8.43s	remaining: 6.43s
454:	learn: 

537:	learn: 0.0008670	test: 0.0276340	best: 0.0227639 (158)	total: 9.93s	remaining: 4.83s
538:	learn: 0.0008658	test: 0.0276543	best: 0.0227639 (158)	total: 9.95s	remaining: 4.82s
539:	learn: 0.0008651	test: 0.0276535	best: 0.0227639 (158)	total: 9.96s	remaining: 4.8s
540:	learn: 0.0008616	test: 0.0276797	best: 0.0227639 (158)	total: 9.98s	remaining: 4.78s
541:	learn: 0.0008586	test: 0.0276984	best: 0.0227639 (158)	total: 10s	remaining: 4.76s
542:	learn: 0.0008516	test: 0.0277655	best: 0.0227639 (158)	total: 10s	remaining: 4.74s
543:	learn: 0.0008447	test: 0.0278132	best: 0.0227639 (158)	total: 10s	remaining: 4.72s
544:	learn: 0.0008425	test: 0.0278355	best: 0.0227639 (158)	total: 10s	remaining: 4.7s
545:	learn: 0.0008419	test: 0.0278447	best: 0.0227639 (158)	total: 10.1s	remaining: 4.68s
546:	learn: 0.0008357	test: 0.0278807	best: 0.0227639 (158)	total: 10.1s	remaining: 4.66s
547:	learn: 0.0008348	test: 0.0278821	best: 0.0227639 (158)	total: 10.1s	remaining: 4.64s
548:	learn: 0.000833

632:	learn: 0.0006687	test: 0.0283641	best: 0.0227639 (158)	total: 11.6s	remaining: 3.06s
633:	learn: 0.0006682	test: 0.0283513	best: 0.0227639 (158)	total: 11.6s	remaining: 3.04s
634:	learn: 0.0006660	test: 0.0284105	best: 0.0227639 (158)	total: 11.6s	remaining: 3.03s
635:	learn: 0.0006643	test: 0.0284141	best: 0.0227639 (158)	total: 11.7s	remaining: 3.02s
636:	learn: 0.0006634	test: 0.0284206	best: 0.0227639 (158)	total: 11.7s	remaining: 3s
637:	learn: 0.0006618	test: 0.0284155	best: 0.0227639 (158)	total: 11.7s	remaining: 2.98s
638:	learn: 0.0006606	test: 0.0284180	best: 0.0227639 (158)	total: 11.8s	remaining: 2.96s
639:	learn: 0.0006590	test: 0.0283926	best: 0.0227639 (158)	total: 11.8s	remaining: 2.94s
640:	learn: 0.0006574	test: 0.0284152	best: 0.0227639 (158)	total: 11.8s	remaining: 2.92s
641:	learn: 0.0006571	test: 0.0284312	best: 0.0227639 (158)	total: 11.8s	remaining: 2.91s
642:	learn: 0.0006532	test: 0.0284192	best: 0.0227639 (158)	total: 11.8s	remaining: 2.89s
643:	learn: 0

725:	learn: 0.0005434	test: 0.0289493	best: 0.0227639 (158)	total: 13.3s	remaining: 1.35s
726:	learn: 0.0005431	test: 0.0289478	best: 0.0227639 (158)	total: 13.3s	remaining: 1.33s
727:	learn: 0.0005425	test: 0.0289284	best: 0.0227639 (158)	total: 13.3s	remaining: 1.31s
728:	learn: 0.0005420	test: 0.0289243	best: 0.0227639 (158)	total: 13.3s	remaining: 1.3s
729:	learn: 0.0005417	test: 0.0289302	best: 0.0227639 (158)	total: 13.3s	remaining: 1.28s
730:	learn: 0.0005411	test: 0.0289358	best: 0.0227639 (158)	total: 13.3s	remaining: 1.26s
731:	learn: 0.0005407	test: 0.0289368	best: 0.0227639 (158)	total: 13.4s	remaining: 1.24s
732:	learn: 0.0005395	test: 0.0289352	best: 0.0227639 (158)	total: 13.4s	remaining: 1.22s
733:	learn: 0.0005387	test: 0.0289236	best: 0.0227639 (158)	total: 13.4s	remaining: 1.2s
734:	learn: 0.0005382	test: 0.0289170	best: 0.0227639 (158)	total: 13.4s	remaining: 1.19s
735:	learn: 0.0005375	test: 0.0288961	best: 0.0227639 (158)	total: 13.4s	remaining: 1.17s
736:	learn: 

(0.01731575639403704, 0.9994513775497162)

In [106]:
base_model_evaluate(CatBoostClassifier(**cat_params_B, use_best_model=False), x_train_new_B, y_train_B,early_stop=True)

0:	learn: 0.6604971	test: 0.6608218	best: 0.6608218 (0)	total: 36.6ms	remaining: 27.4s
1:	learn: 0.6292279	test: 0.6298302	best: 0.6298302 (1)	total: 75.6ms	remaining: 28.3s
2:	learn: 0.5998451	test: 0.6004339	best: 0.6004339 (2)	total: 92.4ms	remaining: 23s
3:	learn: 0.5737463	test: 0.5745549	best: 0.5745549 (3)	total: 107ms	remaining: 20s
4:	learn: 0.5502972	test: 0.5517446	best: 0.5517446 (4)	total: 120ms	remaining: 17.9s
5:	learn: 0.5282391	test: 0.5299183	best: 0.5299183 (5)	total: 133ms	remaining: 16.5s
6:	learn: 0.5088559	test: 0.5104936	best: 0.5104936 (6)	total: 146ms	remaining: 15.5s
7:	learn: 0.4895530	test: 0.4917864	best: 0.4917864 (7)	total: 160ms	remaining: 14.8s
8:	learn: 0.4733531	test: 0.4757508	best: 0.4757508 (8)	total: 178ms	remaining: 14.7s
9:	learn: 0.4559859	test: 0.4585282	best: 0.4585282 (9)	total: 190ms	remaining: 14.1s
10:	learn: 0.4394137	test: 0.4422639	best: 0.4422639 (10)	total: 203ms	remaining: 13.6s
11:	learn: 0.4230601	test: 0.4267228	best: 0.4267228 

100:	learn: 0.2110465	test: 0.2377604	best: 0.2377604 (100)	total: 2.06s	remaining: 13.2s
101:	learn: 0.2103971	test: 0.2372582	best: 0.2372582 (101)	total: 2.07s	remaining: 13.2s
102:	learn: 0.2099246	test: 0.2370585	best: 0.2370585 (102)	total: 2.08s	remaining: 13.1s
103:	learn: 0.2096034	test: 0.2366296	best: 0.2366296 (103)	total: 2.1s	remaining: 13s
104:	learn: 0.2093770	test: 0.2363139	best: 0.2363139 (104)	total: 2.11s	remaining: 13s
105:	learn: 0.2089276	test: 0.2359479	best: 0.2359479 (105)	total: 2.13s	remaining: 12.9s
106:	learn: 0.2086685	test: 0.2357314	best: 0.2357314 (106)	total: 2.14s	remaining: 12.9s
107:	learn: 0.2082748	test: 0.2356524	best: 0.2356524 (107)	total: 2.15s	remaining: 12.8s
108:	learn: 0.2077671	test: 0.2352757	best: 0.2352757 (108)	total: 2.16s	remaining: 12.7s
109:	learn: 0.2073368	test: 0.2351794	best: 0.2351794 (109)	total: 2.18s	remaining: 12.7s
110:	learn: 0.2070591	test: 0.2349623	best: 0.2349623 (110)	total: 2.19s	remaining: 12.6s
111:	learn: 0.2

196:	learn: 0.1841723	test: 0.2274086	best: 0.2274086 (196)	total: 3.75s	remaining: 10.5s
197:	learn: 0.1839476	test: 0.2274020	best: 0.2274020 (197)	total: 3.8s	remaining: 10.6s
198:	learn: 0.1836803	test: 0.2273623	best: 0.2273623 (198)	total: 3.81s	remaining: 10.5s
199:	learn: 0.1834598	test: 0.2273957	best: 0.2273623 (198)	total: 3.82s	remaining: 10.5s
200:	learn: 0.1832805	test: 0.2273174	best: 0.2273174 (200)	total: 3.84s	remaining: 10.5s
201:	learn: 0.1830808	test: 0.2274193	best: 0.2273174 (200)	total: 3.85s	remaining: 10.4s
202:	learn: 0.1829907	test: 0.2275164	best: 0.2273174 (200)	total: 3.86s	remaining: 10.4s
203:	learn: 0.1827994	test: 0.2272800	best: 0.2272800 (203)	total: 3.88s	remaining: 10.4s
204:	learn: 0.1825267	test: 0.2271459	best: 0.2271459 (204)	total: 3.9s	remaining: 10.4s
205:	learn: 0.1823394	test: 0.2271691	best: 0.2271459 (204)	total: 3.92s	remaining: 10.4s
206:	learn: 0.1820443	test: 0.2270117	best: 0.2270117 (206)	total: 3.94s	remaining: 10.3s
207:	learn: 

KeyboardInterrupt: 

# Model

In [172]:
clf1_A = LGBMClassifier(**lgb_params_A)
clf2_A = XGBClassifier(**xgb_params_A)
clf3_A = CatBoostClassifier(**cat_params_A)
clf4_A = LogisticRegression(**lr_params_A)

clf1_B = LGBMClassifier(**lgb_params_B)
clf2_B = XGBClassifier(**xgb_params_B)
clf3_B = CatBoostClassifier(**cat_params_B)
clf4_B = LogisticRegression()

clf1_C = LGBMClassifier(**lgb_params_C)
clf2_C = XGBClassifier(**xgb_params_C)
clf3_C = CatBoostClassifier(**cat_params_C)
clf4_C = LogisticRegression(**lr_params_C)

model_A = VotingClassifier(estimators=[('lgb', clf1_A), ('xgb', clf2_A), ('cat', clf3_A), ('lr', clf4_A)], voting='soft', weights=[1,1,3,1])
model_B = VotingClassifier(estimators=[('lgb', clf1_B), ('xgb', clf2_B), ('cat', clf3_B), ('lr', clf4_B)], voting='soft', weights=[1,3,1,1])
model_C = VotingClassifier(estimators=[('lgb', clf1_C), ('xgb', clf2_C), ('cat', clf3_C), ('lr', clf4_C)], voting='soft', weights=[1,3,1,1])

model_A.fit(x_train_new_A, y_train_A)
model_B.fit(x_train_new_B, y_train_B)
model_C.fit(x_train_new_C, y_train_C)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6607993	total: 74.1ms	remaining: 55.5s
1:	learn: 0.6399876	total: 130ms	remaining: 48.6s
2:	learn: 0.6172403	total: 169ms	remaining: 42.1s
3:	learn: 0.5955290	total: 191ms	remaining: 35.7s
4:	learn: 0.5775424	total: 214ms	remaining: 31.9s
5:	learn: 0.5618544	total: 236ms	remaining: 29.3s
6:	learn: 0.5488064	total: 259ms	remaining: 27.5s
7:	learn: 0.5358204	total: 291ms	remaining: 27s
8:	learn: 0.5233097	total: 337ms	remaining: 27.7s
9:	learn: 0.5120086	total: 386ms	remaining: 28.5s
10:	learn: 0.5013990	total: 409ms	remaining: 27.4s
11:	learn: 0.4909499	total: 431ms	remaining: 26.5s
12:	learn: 0.4819493	total: 453ms	remaining: 25.7s
13:	learn: 0.4730457	total: 474ms	remaining: 24.9s
14:	learn: 0.4649812	total: 505ms	remaining: 24.8s
15:	learn: 0.4580833	total: 552ms	remaining: 25.3s
16:	learn: 0.4514146	total: 586ms	remaining: 25.2s
17:	learn: 0.4455762	total: 609ms	remaining: 24.8s
18:	learn: 0.4406490	total: 631ms	remaining: 24.3s
19:	learn: 0.4354185	total: 653ms	remainin

166:	learn: 0.2699670	total: 4.52s	remaining: 15.8s
167:	learn: 0.2696666	total: 4.55s	remaining: 15.8s
168:	learn: 0.2693210	total: 4.57s	remaining: 15.7s
169:	learn: 0.2690417	total: 4.59s	remaining: 15.7s
170:	learn: 0.2686948	total: 4.61s	remaining: 15.6s
171:	learn: 0.2683577	total: 4.63s	remaining: 15.6s
172:	learn: 0.2681200	total: 4.66s	remaining: 15.5s
173:	learn: 0.2677973	total: 4.68s	remaining: 15.5s
174:	learn: 0.2674689	total: 4.7s	remaining: 15.4s
175:	learn: 0.2670938	total: 4.72s	remaining: 15.4s
176:	learn: 0.2666852	total: 4.77s	remaining: 15.4s
177:	learn: 0.2663608	total: 4.79s	remaining: 15.4s
178:	learn: 0.2659171	total: 4.82s	remaining: 15.4s
179:	learn: 0.2656262	total: 4.84s	remaining: 15.3s
180:	learn: 0.2651753	total: 4.86s	remaining: 15.3s
181:	learn: 0.2648986	total: 4.88s	remaining: 15.2s
182:	learn: 0.2645744	total: 4.9s	remaining: 15.2s
183:	learn: 0.2642328	total: 4.92s	remaining: 15.1s
184:	learn: 0.2638547	total: 4.95s	remaining: 15.1s
185:	learn: 0.

332:	learn: 0.2285447	total: 8.4s	remaining: 10.5s
333:	learn: 0.2283285	total: 8.43s	remaining: 10.5s
334:	learn: 0.2281429	total: 8.46s	remaining: 10.5s
335:	learn: 0.2279095	total: 8.48s	remaining: 10.4s
336:	learn: 0.2277844	total: 8.5s	remaining: 10.4s
337:	learn: 0.2275425	total: 8.52s	remaining: 10.4s
338:	learn: 0.2273417	total: 8.54s	remaining: 10.4s
339:	learn: 0.2271126	total: 8.56s	remaining: 10.3s
340:	learn: 0.2268689	total: 8.59s	remaining: 10.3s
341:	learn: 0.2267667	total: 8.63s	remaining: 10.3s
342:	learn: 0.2265598	total: 8.65s	remaining: 10.3s
343:	learn: 0.2264013	total: 8.68s	remaining: 10.2s
344:	learn: 0.2262918	total: 8.71s	remaining: 10.2s
345:	learn: 0.2261431	total: 8.73s	remaining: 10.2s
346:	learn: 0.2260022	total: 8.76s	remaining: 10.2s
347:	learn: 0.2257732	total: 8.78s	remaining: 10.1s
348:	learn: 0.2256517	total: 8.8s	remaining: 10.1s
349:	learn: 0.2254678	total: 8.82s	remaining: 10.1s
350:	learn: 0.2251810	total: 8.85s	remaining: 10.1s
351:	learn: 0.2

496:	learn: 0.2027992	total: 12.2s	remaining: 6.23s
497:	learn: 0.2026897	total: 12.3s	remaining: 6.21s
498:	learn: 0.2025420	total: 12.3s	remaining: 6.18s
499:	learn: 0.2024003	total: 12.3s	remaining: 6.16s
500:	learn: 0.2023542	total: 12.3s	remaining: 6.13s
501:	learn: 0.2022366	total: 12.4s	remaining: 6.11s
502:	learn: 0.2020399	total: 12.4s	remaining: 6.08s
503:	learn: 0.2019460	total: 12.4s	remaining: 6.05s
504:	learn: 0.2017988	total: 12.4s	remaining: 6.03s
505:	learn: 0.2016521	total: 12.5s	remaining: 6.01s
506:	learn: 0.2014785	total: 12.5s	remaining: 5.99s
507:	learn: 0.2012984	total: 12.5s	remaining: 5.96s
508:	learn: 0.2011270	total: 12.5s	remaining: 5.93s
509:	learn: 0.2010079	total: 12.6s	remaining: 5.91s
510:	learn: 0.2008222	total: 12.6s	remaining: 5.88s
511:	learn: 0.2007467	total: 12.6s	remaining: 5.86s
512:	learn: 0.2005549	total: 12.6s	remaining: 5.83s
513:	learn: 0.2004287	total: 12.6s	remaining: 5.81s
514:	learn: 0.2003343	total: 12.7s	remaining: 5.79s
515:	learn: 

...

165:	learn: 0.0127343	total: 2.43s	remaining: 7.83s
166:	learn: 0.0127135	total: 2.45s	remaining: 7.83s
167:	learn: 0.0126767	total: 2.47s	remaining: 7.81s
168:	learn: 0.0126344	total: 2.48s	remaining: 7.79s
169:	learn: 0.0126059	total: 2.49s	remaining: 7.77s
170:	learn: 0.0125644	total: 2.5s	remaining: 7.75s
171:	learn: 0.0125496	total: 2.52s	remaining: 7.72s
172:	learn: 0.0125208	total: 2.53s	remaining: 7.7s
173:	learn: 0.0124933	total: 2.54s	remaining: 7.68s
174:	learn: 0.0124702	total: 2.55s	remaining: 7.66s
175:	learn: 0.0123988	total: 2.57s	remaining: 7.64s
176:	learn: 0.0123558	total: 2.58s	remaining: 7.62s
177:	learn: 0.0123161	total: 2.59s	remaining: 7.6s
178:	learn: 0.0122968	total: 2.6s	remaining: 7.58s
179:	learn: 0.0122741	total: 2.62s	remaining: 7.55s
180:	learn: 0.0122144	total: 2.63s	remaining: 7.53s
181:	learn: 0.0122001	total: 2.66s	remaining: 7.57s
182:	learn: 0.0121466	total: 2.68s	remaining: 7.56s
183:	learn: 0.0120999	total: 2.69s	remaining: 7.54s
184:	learn: 0.01

338:	learn: 0.0083706	total: 4.81s	remaining: 5.12s
339:	learn: 0.0083611	total: 4.84s	remaining: 5.12s
340:	learn: 0.0083515	total: 4.85s	remaining: 5.11s
341:	learn: 0.0083392	total: 4.86s	remaining: 5.09s
342:	learn: 0.0083311	total: 4.88s	remaining: 5.07s
343:	learn: 0.0083173	total: 4.89s	remaining: 5.06s
344:	learn: 0.0083098	total: 4.9s	remaining: 5.04s
345:	learn: 0.0082892	total: 4.91s	remaining: 5.03s
346:	learn: 0.0082695	total: 4.93s	remaining: 5.01s
347:	learn: 0.0082550	total: 4.94s	remaining: 5s
348:	learn: 0.0082230	total: 4.95s	remaining: 4.98s
349:	learn: 0.0082108	total: 4.96s	remaining: 4.96s
350:	learn: 0.0082100	total: 4.97s	remaining: 4.95s
351:	learn: 0.0081965	total: 4.99s	remaining: 4.93s
352:	learn: 0.0081782	total: 5s	remaining: 4.91s
353:	learn: 0.0081439	total: 5.03s	remaining: 4.91s
354:	learn: 0.0081205	total: 5.05s	remaining: 4.91s
355:	learn: 0.0081124	total: 5.06s	remaining: 4.89s
356:	learn: 0.0081008	total: 5.07s	remaining: 4.87s
357:	learn: 0.00809

511:	learn: 0.0063741	total: 7.18s	remaining: 2.64s
512:	learn: 0.0063685	total: 7.2s	remaining: 2.63s
513:	learn: 0.0063669	total: 7.21s	remaining: 2.61s
514:	learn: 0.0063643	total: 7.23s	remaining: 2.6s
515:	learn: 0.0063013	total: 7.24s	remaining: 2.58s
516:	learn: 0.0062993	total: 7.25s	remaining: 2.57s
517:	learn: 0.0062921	total: 7.27s	remaining: 2.55s
518:	learn: 0.0062790	total: 7.28s	remaining: 2.54s
519:	learn: 0.0062660	total: 7.29s	remaining: 2.52s
520:	learn: 0.0062622	total: 7.3s	remaining: 2.51s
521:	learn: 0.0062606	total: 7.32s	remaining: 2.49s
522:	learn: 0.0062594	total: 7.33s	remaining: 2.48s
523:	learn: 0.0062524	total: 7.34s	remaining: 2.46s
524:	learn: 0.0062359	total: 7.35s	remaining: 2.45s
525:	learn: 0.0062308	total: 7.37s	remaining: 2.44s
526:	learn: 0.0062285	total: 7.38s	remaining: 2.42s
527:	learn: 0.0062137	total: 7.41s	remaining: 2.41s
528:	learn: 0.0062062	total: 7.43s	remaining: 2.4s
529:	learn: 0.0061675	total: 7.44s	remaining: 2.38s
530:	learn: 0.00

671:	learn: 0.0052439	total: 9.39s	remaining: 391ms
672:	learn: 0.0052421	total: 9.4s	remaining: 377ms
673:	learn: 0.0052399	total: 9.42s	remaining: 363ms
674:	learn: 0.0052311	total: 9.43s	remaining: 349ms
675:	learn: 0.0052301	total: 9.44s	remaining: 335ms
676:	learn: 0.0052165	total: 9.46s	remaining: 321ms
677:	learn: 0.0052097	total: 9.47s	remaining: 307ms
678:	learn: 0.0052078	total: 9.48s	remaining: 293ms
679:	learn: 0.0051941	total: 9.49s	remaining: 279ms
680:	learn: 0.0051868	total: 9.51s	remaining: 265ms
681:	learn: 0.0051811	total: 9.52s	remaining: 251ms
682:	learn: 0.0051685	total: 9.53s	remaining: 237ms
683:	learn: 0.0051681	total: 9.54s	remaining: 223ms
684:	learn: 0.0051672	total: 9.56s	remaining: 209ms
685:	learn: 0.0051627	total: 9.57s	remaining: 195ms
686:	learn: 0.0051613	total: 9.58s	remaining: 181ms
687:	learn: 0.0051607	total: 9.61s	remaining: 168ms
688:	learn: 0.0051464	total: 9.63s	remaining: 154ms
689:	learn: 0.0051449	total: 9.64s	remaining: 140ms
690:	learn: 0

VotingClassifier(estimators=[('lgb', LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.05, max_depth=4, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_iterations=150, num_leaves=31, objective=None,...ty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='soft',
         weights=[1, 3, 1, 1])

In [173]:
base_model_evaluate(model_A, x_train_new_A, y_train_A)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6661357	total: 60.8ms	remaining: 45.5s
1:	learn: 0.6429231	total: 119ms	remaining: 44.5s
2:	learn: 0.6184447	total: 163ms	remaining: 40.6s
3:	learn: 0.5981031	total: 189ms	remaining: 35.2s
4:	learn: 0.5794217	total: 210ms	remaining: 31.3s
5:	learn: 0.5629191	total: 231ms	remaining: 28.6s
6:	learn: 0.5493596	total: 250ms	remaining: 26.5s
7:	learn: 0.5375462	total: 290ms	remaining: 26.9s
8:	learn: 0.5253257	total: 336ms	remaining: 27.7s
9:	learn: 0.5139399	total: 368ms	remaining: 27.2s
10:	learn: 0.5043136	total: 396ms	remaining: 26.6s
11:	learn: 0.4953474	total: 434ms	remaining: 26.7s
12:	learn: 0.4861783	total: 465ms	remaining: 26.4s
13:	learn: 0.4787947	total: 485ms	remaining: 25.5s
14:	learn: 0.4721040	total: 526ms	remaining: 25.8s
15:	learn: 0.4647277	total: 569ms	remaining: 26.1s
16:	learn: 0.4578142	total: 591ms	remaining: 25.5s
17:	learn: 0.4506476	total: 612ms	remaining: 24.9s
18:	learn: 0.4451112	total: 632ms	remaining: 24.3s
19:	learn: 0.4397728	total: 651ms	remain

168:	learn: 0.2667401	total: 4.35s	remaining: 15s
169:	learn: 0.2663532	total: 4.37s	remaining: 14.9s
170:	learn: 0.2659779	total: 4.39s	remaining: 14.9s
171:	learn: 0.2656621	total: 4.42s	remaining: 14.8s
172:	learn: 0.2653381	total: 4.43s	remaining: 14.8s
173:	learn: 0.2648659	total: 4.46s	remaining: 14.7s
174:	learn: 0.2645722	total: 4.48s	remaining: 14.7s
175:	learn: 0.2641485	total: 4.5s	remaining: 14.7s
176:	learn: 0.2637993	total: 4.52s	remaining: 14.6s
177:	learn: 0.2634898	total: 4.54s	remaining: 14.6s
178:	learn: 0.2631701	total: 4.57s	remaining: 14.6s
179:	learn: 0.2627758	total: 4.6s	remaining: 14.6s
180:	learn: 0.2624905	total: 4.62s	remaining: 14.5s
181:	learn: 0.2622239	total: 4.64s	remaining: 14.5s
182:	learn: 0.2618918	total: 4.66s	remaining: 14.4s
183:	learn: 0.2614371	total: 4.68s	remaining: 14.4s
184:	learn: 0.2611481	total: 4.7s	remaining: 14.4s
185:	learn: 0.2607137	total: 4.72s	remaining: 14.3s
186:	learn: 0.2603109	total: 4.75s	remaining: 14.3s
187:	learn: 0.259

328:	learn: 0.2237836	total: 7.89s	remaining: 10.1s
329:	learn: 0.2235423	total: 7.91s	remaining: 10.1s
330:	learn: 0.2233467	total: 7.93s	remaining: 10s
331:	learn: 0.2230867	total: 7.95s	remaining: 10s
332:	learn: 0.2228613	total: 7.97s	remaining: 9.98s
333:	learn: 0.2226740	total: 7.99s	remaining: 9.95s
334:	learn: 0.2224037	total: 8.01s	remaining: 9.93s
335:	learn: 0.2221393	total: 8.03s	remaining: 9.9s
336:	learn: 0.2219533	total: 8.05s	remaining: 9.87s
337:	learn: 0.2218374	total: 8.07s	remaining: 9.84s
338:	learn: 0.2217636	total: 8.11s	remaining: 9.84s
339:	learn: 0.2216848	total: 8.14s	remaining: 9.82s
340:	learn: 0.2215330	total: 8.16s	remaining: 9.79s
341:	learn: 0.2212386	total: 8.18s	remaining: 9.76s
342:	learn: 0.2209896	total: 8.2s	remaining: 9.73s
343:	learn: 0.2208375	total: 8.23s	remaining: 9.71s
344:	learn: 0.2207010	total: 8.25s	remaining: 9.68s
345:	learn: 0.2205447	total: 8.27s	remaining: 9.65s
346:	learn: 0.2203344	total: 8.29s	remaining: 9.62s
347:	learn: 0.2201

489:	learn: 0.1961708	total: 11.5s	remaining: 6.09s
490:	learn: 0.1960664	total: 11.5s	remaining: 6.07s
491:	learn: 0.1959383	total: 11.5s	remaining: 6.04s
492:	learn: 0.1957952	total: 11.5s	remaining: 6.02s
493:	learn: 0.1956439	total: 11.6s	remaining: 5.99s
494:	learn: 0.1955882	total: 11.6s	remaining: 5.97s
495:	learn: 0.1955363	total: 11.6s	remaining: 5.94s
496:	learn: 0.1953905	total: 11.6s	remaining: 5.92s
497:	learn: 0.1952575	total: 11.6s	remaining: 5.89s
498:	learn: 0.1950259	total: 11.7s	remaining: 5.87s
499:	learn: 0.1949079	total: 11.7s	remaining: 5.85s
500:	learn: 0.1947074	total: 11.7s	remaining: 5.83s
501:	learn: 0.1944765	total: 11.7s	remaining: 5.8s
502:	learn: 0.1943884	total: 11.8s	remaining: 5.78s
503:	learn: 0.1942736	total: 11.8s	remaining: 5.75s
504:	learn: 0.1940692	total: 11.8s	remaining: 5.73s
505:	learn: 0.1940254	total: 11.8s	remaining: 5.71s
506:	learn: 0.1938835	total: 11.9s	remaining: 5.68s
507:	learn: 0.1936973	total: 11.9s	remaining: 5.66s
508:	learn: 0

...

163:	learn: 0.2670668	total: 3.87s	remaining: 13.8s
164:	learn: 0.2665457	total: 3.9s	remaining: 13.8s
165:	learn: 0.2660978	total: 3.92s	remaining: 13.8s
166:	learn: 0.2658005	total: 3.94s	remaining: 13.7s
167:	learn: 0.2654438	total: 3.96s	remaining: 13.7s
168:	learn: 0.2650950	total: 3.97s	remaining: 13.7s
169:	learn: 0.2647945	total: 4s	remaining: 13.6s
170:	learn: 0.2644601	total: 4.01s	remaining: 13.6s
171:	learn: 0.2642055	total: 4.03s	remaining: 13.6s
172:	learn: 0.2636794	total: 4.05s	remaining: 13.5s
173:	learn: 0.2633829	total: 4.08s	remaining: 13.5s
174:	learn: 0.2630003	total: 4.11s	remaining: 13.5s
175:	learn: 0.2624766	total: 4.13s	remaining: 13.5s
176:	learn: 0.2621468	total: 4.16s	remaining: 13.5s
177:	learn: 0.2617422	total: 4.18s	remaining: 13.4s
178:	learn: 0.2613485	total: 4.2s	remaining: 13.4s
179:	learn: 0.2610452	total: 4.22s	remaining: 13.4s
180:	learn: 0.2606188	total: 4.24s	remaining: 13.3s
181:	learn: 0.2603178	total: 4.26s	remaining: 13.3s
182:	learn: 0.259

323:	learn: 0.2213226	total: 7.36s	remaining: 9.68s
324:	learn: 0.2210223	total: 7.39s	remaining: 9.67s
325:	learn: 0.2209066	total: 7.41s	remaining: 9.64s
326:	learn: 0.2206160	total: 7.43s	remaining: 9.61s
327:	learn: 0.2203015	total: 7.45s	remaining: 9.59s
328:	learn: 0.2200168	total: 7.47s	remaining: 9.56s
329:	learn: 0.2197059	total: 7.49s	remaining: 9.54s
330:	learn: 0.2193895	total: 7.53s	remaining: 9.53s
331:	learn: 0.2191751	total: 7.55s	remaining: 9.51s
332:	learn: 0.2189396	total: 7.57s	remaining: 9.48s
333:	learn: 0.2187193	total: 7.61s	remaining: 9.47s
334:	learn: 0.2185880	total: 7.63s	remaining: 9.45s
335:	learn: 0.2182806	total: 7.65s	remaining: 9.43s
336:	learn: 0.2180431	total: 7.67s	remaining: 9.4s
337:	learn: 0.2178208	total: 7.69s	remaining: 9.37s
338:	learn: 0.2177139	total: 7.71s	remaining: 9.35s
339:	learn: 0.2176001	total: 7.73s	remaining: 9.32s
340:	learn: 0.2173477	total: 7.75s	remaining: 9.3s
341:	learn: 0.2170819	total: 7.77s	remaining: 9.27s
342:	learn: 0.

484:	learn: 0.1911165	total: 10.9s	remaining: 5.95s
485:	learn: 0.1909213	total: 10.9s	remaining: 5.93s
486:	learn: 0.1908213	total: 10.9s	remaining: 5.91s
487:	learn: 0.1907015	total: 11s	remaining: 5.88s
488:	learn: 0.1905675	total: 11s	remaining: 5.86s
489:	learn: 0.1904824	total: 11s	remaining: 5.84s
490:	learn: 0.1903214	total: 11s	remaining: 5.82s
491:	learn: 0.1901697	total: 11s	remaining: 5.79s
492:	learn: 0.1899736	total: 11.1s	remaining: 5.77s
493:	learn: 0.1897450	total: 11.1s	remaining: 5.75s
494:	learn: 0.1896647	total: 11.1s	remaining: 5.73s
495:	learn: 0.1894705	total: 11.1s	remaining: 5.71s
496:	learn: 0.1893868	total: 11.2s	remaining: 5.68s
497:	learn: 0.1891777	total: 11.2s	remaining: 5.66s
498:	learn: 0.1889849	total: 11.2s	remaining: 5.64s
499:	learn: 0.1888540	total: 11.2s	remaining: 5.62s
500:	learn: 0.1886301	total: 11.3s	remaining: 5.59s
501:	learn: 0.1884521	total: 11.3s	remaining: 5.57s
502:	learn: 0.1882911	total: 11.3s	remaining: 5.54s
503:	learn: 0.1881379	

644:	learn: 0.1682007	total: 14.5s	remaining: 2.35s
645:	learn: 0.1680653	total: 14.5s	remaining: 2.33s
646:	learn: 0.1679414	total: 14.5s	remaining: 2.31s
647:	learn: 0.1678072	total: 14.5s	remaining: 2.29s
648:	learn: 0.1676793	total: 14.5s	remaining: 2.26s
649:	learn: 0.1675575	total: 14.6s	remaining: 2.24s
650:	learn: 0.1674646	total: 14.6s	remaining: 2.22s
651:	learn: 0.1673288	total: 14.6s	remaining: 2.19s
652:	learn: 0.1671624	total: 14.6s	remaining: 2.17s
653:	learn: 0.1669933	total: 14.6s	remaining: 2.15s
654:	learn: 0.1668429	total: 14.7s	remaining: 2.13s
655:	learn: 0.1667902	total: 14.7s	remaining: 2.11s
656:	learn: 0.1666433	total: 14.7s	remaining: 2.08s
657:	learn: 0.1665669	total: 14.7s	remaining: 2.06s
658:	learn: 0.1664378	total: 14.8s	remaining: 2.04s
659:	learn: 0.1663598	total: 14.8s	remaining: 2.02s
660:	learn: 0.1662587	total: 14.8s	remaining: 1.99s
661:	learn: 0.1660898	total: 14.8s	remaining: 1.97s
662:	learn: 0.1659918	total: 14.8s	remaining: 1.95s
663:	learn: 

(0.26682615716954966, 0.9556214019687703)

In [174]:
base_model_evaluate(model_B, x_train_new_B, y_train_B)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6616044	total: 35.4ms	remaining: 24.8s
1:	learn: 0.6307556	total: 75.6ms	remaining: 26.4s
2:	learn: 0.6004330	total: 113ms	remaining: 26.4s
3:	learn: 0.5727694	total: 139ms	remaining: 24.2s
4:	learn: 0.5477139	total: 164ms	remaining: 22.8s
5:	learn: 0.5257824	total: 197ms	remaining: 22.7s
6:	learn: 0.5067828	total: 212ms	remaining: 21s
7:	learn: 0.4864003	total: 223ms	remaining: 19.3s
8:	learn: 0.4698931	total: 235ms	remaining: 18.1s
9:	learn: 0.4531718	total: 266ms	remaining: 18.3s
10:	learn: 0.4370886	total: 295ms	remaining: 18.5s
11:	learn: 0.4239909	total: 324ms	remaining: 18.6s
12:	learn: 0.4104250	total: 336ms	remaining: 17.8s
13:	learn: 0.3983656	total: 348ms	remaining: 17.1s
14:	learn: 0.3885408	total: 360ms	remaining: 16.4s
15:	learn: 0.3767983	total: 372ms	remaining: 15.9s
16:	learn: 0.3671846	total: 382ms	remaining: 15.4s
17:	learn: 0.3583176	total: 394ms	remaining: 14.9s
18:	learn: 0.3489831	total: 406ms	remaining: 14.5s
19:	learn: 0.3409520	total: 416ms	remaini

170:	learn: 0.1881563	total: 2.44s	remaining: 7.54s
171:	learn: 0.1878133	total: 2.46s	remaining: 7.54s
172:	learn: 0.1876561	total: 2.47s	remaining: 7.52s
173:	learn: 0.1874165	total: 2.48s	remaining: 7.5s
174:	learn: 0.1872843	total: 2.49s	remaining: 7.48s
175:	learn: 0.1871319	total: 2.5s	remaining: 7.46s
176:	learn: 0.1869741	total: 2.52s	remaining: 7.43s
177:	learn: 0.1867356	total: 2.53s	remaining: 7.42s
178:	learn: 0.1865504	total: 2.54s	remaining: 7.39s
179:	learn: 0.1863819	total: 2.55s	remaining: 7.37s
180:	learn: 0.1861687	total: 2.56s	remaining: 7.34s
181:	learn: 0.1859931	total: 2.57s	remaining: 7.32s
182:	learn: 0.1857813	total: 2.58s	remaining: 7.3s
183:	learn: 0.1856214	total: 2.6s	remaining: 7.28s
184:	learn: 0.1854693	total: 2.61s	remaining: 7.25s
185:	learn: 0.1851639	total: 2.62s	remaining: 7.23s
186:	learn: 0.1850529	total: 2.63s	remaining: 7.21s
187:	learn: 0.1847997	total: 2.65s	remaining: 7.21s
188:	learn: 0.1845272	total: 2.67s	remaining: 7.22s
189:	learn: 0.18

339:	learn: 0.1594862	total: 4.56s	remaining: 4.83s
340:	learn: 0.1593876	total: 4.59s	remaining: 4.83s
341:	learn: 0.1592919	total: 4.6s	remaining: 4.82s
342:	learn: 0.1591509	total: 4.61s	remaining: 4.8s
343:	learn: 0.1589933	total: 4.62s	remaining: 4.79s
344:	learn: 0.1588628	total: 4.64s	remaining: 4.77s
345:	learn: 0.1587993	total: 4.65s	remaining: 4.76s
346:	learn: 0.1587100	total: 4.66s	remaining: 4.74s
347:	learn: 0.1585080	total: 4.67s	remaining: 4.73s
348:	learn: 0.1583395	total: 4.68s	remaining: 4.71s
349:	learn: 0.1582483	total: 4.7s	remaining: 4.7s
350:	learn: 0.1580972	total: 4.71s	remaining: 4.68s
351:	learn: 0.1579657	total: 4.72s	remaining: 4.67s
352:	learn: 0.1578791	total: 4.73s	remaining: 4.65s
353:	learn: 0.1577564	total: 4.74s	remaining: 4.64s
354:	learn: 0.1577094	total: 4.75s	remaining: 4.62s
355:	learn: 0.1575461	total: 4.78s	remaining: 4.62s
356:	learn: 0.1573859	total: 4.8s	remaining: 4.61s
357:	learn: 0.1572091	total: 4.81s	remaining: 4.59s
358:	learn: 0.157

507:	learn: 0.1396660	total: 7.19s	remaining: 2.72s
508:	learn: 0.1395540	total: 7.2s	remaining: 2.7s
509:	learn: 0.1394800	total: 7.21s	remaining: 2.69s
510:	learn: 0.1393837	total: 7.22s	remaining: 2.67s
511:	learn: 0.1393183	total: 7.24s	remaining: 2.66s
512:	learn: 0.1392147	total: 7.25s	remaining: 2.64s
513:	learn: 0.1391391	total: 7.26s	remaining: 2.63s
514:	learn: 0.1390394	total: 7.27s	remaining: 2.61s
515:	learn: 0.1390055	total: 7.28s	remaining: 2.6s
516:	learn: 0.1389199	total: 7.29s	remaining: 2.58s
517:	learn: 0.1387909	total: 7.3s	remaining: 2.57s
518:	learn: 0.1386599	total: 7.31s	remaining: 2.55s
519:	learn: 0.1385274	total: 7.33s	remaining: 2.54s
520:	learn: 0.1383714	total: 7.34s	remaining: 2.52s
521:	learn: 0.1383221	total: 7.36s	remaining: 2.51s
522:	learn: 0.1382221	total: 7.37s	remaining: 2.5s
523:	learn: 0.1381226	total: 7.39s	remaining: 2.48s
524:	learn: 0.1380958	total: 7.41s	remaining: 2.47s
525:	learn: 0.1379905	total: 7.43s	remaining: 2.46s
526:	learn: 0.137

...

162:	learn: 0.1991784	total: 2.25s	remaining: 7.42s
163:	learn: 0.1989123	total: 2.27s	remaining: 7.41s
164:	learn: 0.1986422	total: 2.28s	remaining: 7.39s
165:	learn: 0.1983245	total: 2.29s	remaining: 7.37s
166:	learn: 0.1981632	total: 2.3s	remaining: 7.35s
167:	learn: 0.1978825	total: 2.31s	remaining: 7.32s
168:	learn: 0.1977287	total: 2.32s	remaining: 7.3s
169:	learn: 0.1974807	total: 2.33s	remaining: 7.28s
170:	learn: 0.1972404	total: 2.35s	remaining: 7.26s
171:	learn: 0.1970667	total: 2.36s	remaining: 7.24s
172:	learn: 0.1968898	total: 2.37s	remaining: 7.22s
173:	learn: 0.1967153	total: 2.38s	remaining: 7.2s
174:	learn: 0.1966561	total: 2.39s	remaining: 7.18s
175:	learn: 0.1964465	total: 2.4s	remaining: 7.16s
176:	learn: 0.1961019	total: 2.42s	remaining: 7.14s
177:	learn: 0.1959802	total: 2.43s	remaining: 7.12s
178:	learn: 0.1957996	total: 2.44s	remaining: 7.1s
179:	learn: 0.1956176	total: 2.45s	remaining: 7.08s
180:	learn: 0.1954446	total: 2.48s	remaining: 7.1s
181:	learn: 0.1952

334:	learn: 0.1687046	total: 4.39s	remaining: 4.78s
335:	learn: 0.1685226	total: 4.42s	remaining: 4.79s
336:	learn: 0.1683364	total: 4.43s	remaining: 4.77s
337:	learn: 0.1681305	total: 4.44s	remaining: 4.76s
338:	learn: 0.1679856	total: 4.45s	remaining: 4.74s
339:	learn: 0.1677931	total: 4.46s	remaining: 4.73s
340:	learn: 0.1676092	total: 4.48s	remaining: 4.71s
341:	learn: 0.1675053	total: 4.49s	remaining: 4.7s
342:	learn: 0.1674594	total: 4.5s	remaining: 4.68s
343:	learn: 0.1673183	total: 4.51s	remaining: 4.67s
344:	learn: 0.1672750	total: 4.52s	remaining: 4.66s
345:	learn: 0.1671483	total: 4.54s	remaining: 4.64s
346:	learn: 0.1670708	total: 4.55s	remaining: 4.63s
347:	learn: 0.1669722	total: 4.56s	remaining: 4.61s
348:	learn: 0.1668997	total: 4.57s	remaining: 4.6s
349:	learn: 0.1667683	total: 4.58s	remaining: 4.58s
350:	learn: 0.1666493	total: 4.61s	remaining: 4.58s
351:	learn: 0.1664731	total: 4.63s	remaining: 4.58s
352:	learn: 0.1663732	total: 4.64s	remaining: 4.56s
353:	learn: 0.1

504:	learn: 0.1478131	total: 6.54s	remaining: 2.52s
505:	learn: 0.1476805	total: 6.55s	remaining: 2.51s
506:	learn: 0.1476125	total: 6.57s	remaining: 2.5s
507:	learn: 0.1475382	total: 6.58s	remaining: 2.49s
508:	learn: 0.1474339	total: 6.59s	remaining: 2.47s
509:	learn: 0.1473243	total: 6.6s	remaining: 2.46s
510:	learn: 0.1471135	total: 6.61s	remaining: 2.44s
511:	learn: 0.1470225	total: 6.62s	remaining: 2.43s
512:	learn: 0.1468258	total: 6.63s	remaining: 2.42s
513:	learn: 0.1467210	total: 6.65s	remaining: 2.4s
514:	learn: 0.1466462	total: 6.66s	remaining: 2.39s
515:	learn: 0.1465793	total: 6.67s	remaining: 2.38s
516:	learn: 0.1464952	total: 6.68s	remaining: 2.36s
517:	learn: 0.1464550	total: 6.69s	remaining: 2.35s
518:	learn: 0.1464090	total: 6.7s	remaining: 2.34s
519:	learn: 0.1462514	total: 6.72s	remaining: 2.32s
520:	learn: 0.1461106	total: 6.73s	remaining: 2.31s
521:	learn: 0.1459972	total: 6.74s	remaining: 2.3s
522:	learn: 0.1459669	total: 6.77s	remaining: 2.29s
523:	learn: 0.145

675:	learn: 0.1313792	total: 8.65s	remaining: 307ms
676:	learn: 0.1313047	total: 8.68s	remaining: 295ms
677:	learn: 0.1312374	total: 8.69s	remaining: 282ms
678:	learn: 0.1310688	total: 8.7s	remaining: 269ms
679:	learn: 0.1309467	total: 8.71s	remaining: 256ms
680:	learn: 0.1309044	total: 8.72s	remaining: 243ms
681:	learn: 0.1307936	total: 8.73s	remaining: 231ms
682:	learn: 0.1306845	total: 8.74s	remaining: 218ms
683:	learn: 0.1306207	total: 8.76s	remaining: 205ms
684:	learn: 0.1305625	total: 8.77s	remaining: 192ms
685:	learn: 0.1305078	total: 8.78s	remaining: 179ms
686:	learn: 0.1303988	total: 8.79s	remaining: 166ms
687:	learn: 0.1302577	total: 8.8s	remaining: 154ms
688:	learn: 0.1301932	total: 8.81s	remaining: 141ms
689:	learn: 0.1301065	total: 8.82s	remaining: 128ms
690:	learn: 0.1300134	total: 8.83s	remaining: 115ms
691:	learn: 0.1299738	total: 8.85s	remaining: 102ms
692:	learn: 0.1298902	total: 8.87s	remaining: 89.6ms
693:	learn: 0.1298358	total: 8.89s	remaining: 76.8ms
694:	learn: 

(0.19127842109299703, 0.8830857883918958)

In [175]:
base_model_evaluate(model_C, x_train_new_C, y_train_C)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.5859885	total: 28.9ms	remaining: 20.2s
1:	learn: 0.4912923	total: 61.4ms	remaining: 21.4s
2:	learn: 0.4314617	total: 97.5ms	remaining: 22.6s
3:	learn: 0.3664017	total: 126ms	remaining: 22s
4:	learn: 0.3135279	total: 144ms	remaining: 20s
5:	learn: 0.2698691	total: 158ms	remaining: 18.3s
6:	learn: 0.2346936	total: 184ms	remaining: 18.2s
7:	learn: 0.2082644	total: 195ms	remaining: 16.8s
8:	learn: 0.1823718	total: 215ms	remaining: 16.5s
9:	learn: 0.1611896	total: 224ms	remaining: 15.5s
10:	learn: 0.1434121	total: 249ms	remaining: 15.6s
11:	learn: 0.1284983	total: 275ms	remaining: 15.8s
12:	learn: 0.1159083	total: 305ms	remaining: 16.1s
13:	learn: 0.1055686	total: 318ms	remaining: 15.6s
14:	learn: 0.0968795	total: 328ms	remaining: 15s
15:	learn: 0.0889018	total: 338ms	remaining: 14.4s
16:	learn: 0.0817235	total: 350ms	remaining: 14.1s
17:	learn: 0.0740412	total: 360ms	remaining: 13.7s
18:	learn: 0.0702288	total: 372ms	remaining: 13.3s
19:	learn: 0.0666409	total: 381ms	remaining:

165:	learn: 0.0121386	total: 2.13s	remaining: 6.86s
166:	learn: 0.0121010	total: 2.14s	remaining: 6.84s
167:	learn: 0.0120401	total: 2.15s	remaining: 6.82s
168:	learn: 0.0119508	total: 2.17s	remaining: 6.8s
169:	learn: 0.0119330	total: 2.17s	remaining: 6.78s
170:	learn: 0.0118945	total: 2.18s	remaining: 6.76s
171:	learn: 0.0118191	total: 2.19s	remaining: 6.74s
172:	learn: 0.0117911	total: 2.2s	remaining: 6.71s
173:	learn: 0.0117582	total: 2.21s	remaining: 6.69s
174:	learn: 0.0117367	total: 2.22s	remaining: 6.67s
175:	learn: 0.0116940	total: 2.23s	remaining: 6.64s
176:	learn: 0.0116840	total: 2.24s	remaining: 6.62s
177:	learn: 0.0116338	total: 2.25s	remaining: 6.6s
178:	learn: 0.0116083	total: 2.26s	remaining: 6.58s
179:	learn: 0.0115547	total: 2.27s	remaining: 6.56s
180:	learn: 0.0115381	total: 2.28s	remaining: 6.54s
181:	learn: 0.0114784	total: 2.29s	remaining: 6.52s
182:	learn: 0.0114364	total: 2.3s	remaining: 6.5s
183:	learn: 0.0113929	total: 2.31s	remaining: 6.47s
184:	learn: 0.011

326:	learn: 0.0076899	total: 3.84s	remaining: 4.38s
327:	learn: 0.0076484	total: 3.86s	remaining: 4.38s
328:	learn: 0.0076191	total: 3.88s	remaining: 4.37s
329:	learn: 0.0076103	total: 3.89s	remaining: 4.36s
330:	learn: 0.0075893	total: 3.9s	remaining: 4.34s
331:	learn: 0.0075852	total: 3.91s	remaining: 4.33s
332:	learn: 0.0075649	total: 3.92s	remaining: 4.32s
333:	learn: 0.0075602	total: 3.93s	remaining: 4.3s
334:	learn: 0.0075263	total: 3.94s	remaining: 4.29s
335:	learn: 0.0075170	total: 3.95s	remaining: 4.28s
336:	learn: 0.0075067	total: 3.96s	remaining: 4.26s
337:	learn: 0.0075000	total: 3.97s	remaining: 4.25s
338:	learn: 0.0074934	total: 3.98s	remaining: 4.23s
339:	learn: 0.0074720	total: 3.98s	remaining: 4.22s
340:	learn: 0.0074498	total: 4s	remaining: 4.21s
341:	learn: 0.0074305	total: 4s	remaining: 4.19s
342:	learn: 0.0074086	total: 4.01s	remaining: 4.18s
343:	learn: 0.0073887	total: 4.03s	remaining: 4.17s
344:	learn: 0.0073849	total: 4.04s	remaining: 4.15s
345:	learn: 0.007381

504:	learn: 0.0055347	total: 5.75s	remaining: 2.22s
505:	learn: 0.0055286	total: 5.77s	remaining: 2.21s
506:	learn: 0.0055217	total: 5.79s	remaining: 2.2s
507:	learn: 0.0055083	total: 5.8s	remaining: 2.19s
508:	learn: 0.0055045	total: 5.81s	remaining: 2.18s
509:	learn: 0.0055016	total: 5.82s	remaining: 2.17s
510:	learn: 0.0055008	total: 5.83s	remaining: 2.15s
511:	learn: 0.0054985	total: 5.83s	remaining: 2.14s
512:	learn: 0.0054964	total: 5.84s	remaining: 2.13s
513:	learn: 0.0054748	total: 5.85s	remaining: 2.12s
514:	learn: 0.0054700	total: 5.86s	remaining: 2.11s
515:	learn: 0.0054672	total: 5.87s	remaining: 2.09s
516:	learn: 0.0054506	total: 5.88s	remaining: 2.08s
517:	learn: 0.0054495	total: 5.89s	remaining: 2.07s
518:	learn: 0.0054235	total: 5.9s	remaining: 2.06s
519:	learn: 0.0054126	total: 5.91s	remaining: 2.05s
520:	learn: 0.0054017	total: 5.92s	remaining: 2.04s
521:	learn: 0.0054011	total: 5.93s	remaining: 2.02s
522:	learn: 0.0053838	total: 5.94s	remaining: 2.01s
523:	learn: 0.0

...

321:	learn: 0.0070622	total: 3.63s	remaining: 4.27s
322:	learn: 0.0070470	total: 3.66s	remaining: 4.27s
323:	learn: 0.0070301	total: 3.67s	remaining: 4.26s
324:	learn: 0.0069933	total: 3.68s	remaining: 4.25s
325:	learn: 0.0069798	total: 3.69s	remaining: 4.24s
326:	learn: 0.0069540	total: 3.7s	remaining: 4.22s
327:	learn: 0.0069466	total: 3.71s	remaining: 4.21s
328:	learn: 0.0069283	total: 3.72s	remaining: 4.2s
329:	learn: 0.0068974	total: 3.73s	remaining: 4.18s
330:	learn: 0.0068693	total: 3.74s	remaining: 4.17s
331:	learn: 0.0068667	total: 3.75s	remaining: 4.16s
332:	learn: 0.0068483	total: 3.76s	remaining: 4.15s
333:	learn: 0.0068224	total: 3.77s	remaining: 4.13s
334:	learn: 0.0068072	total: 3.78s	remaining: 4.12s
335:	learn: 0.0067886	total: 3.79s	remaining: 4.11s
336:	learn: 0.0067801	total: 3.8s	remaining: 4.09s
337:	learn: 0.0067677	total: 3.81s	remaining: 4.08s
338:	learn: 0.0067495	total: 3.82s	remaining: 4.07s
339:	learn: 0.0067263	total: 3.83s	remaining: 4.05s
340:	learn: 0.0

500:	learn: 0.0047120	total: 5.58s	remaining: 2.22s
501:	learn: 0.0047027	total: 5.6s	remaining: 2.21s
502:	learn: 0.0047000	total: 5.61s	remaining: 2.2s
503:	learn: 0.0046968	total: 5.62s	remaining: 2.19s
504:	learn: 0.0046733	total: 5.63s	remaining: 2.17s
505:	learn: 0.0046632	total: 5.64s	remaining: 2.16s
506:	learn: 0.0046574	total: 5.65s	remaining: 2.15s
507:	learn: 0.0046566	total: 5.66s	remaining: 2.14s
508:	learn: 0.0046441	total: 5.67s	remaining: 2.13s
509:	learn: 0.0046424	total: 5.68s	remaining: 2.12s
510:	learn: 0.0046382	total: 5.69s	remaining: 2.1s
511:	learn: 0.0046262	total: 5.7s	remaining: 2.09s
512:	learn: 0.0046091	total: 5.71s	remaining: 2.08s
513:	learn: 0.0046075	total: 5.72s	remaining: 2.07s
514:	learn: 0.0046043	total: 5.73s	remaining: 2.06s
515:	learn: 0.0045982	total: 5.74s	remaining: 2.05s
516:	learn: 0.0045918	total: 5.75s	remaining: 2.04s
517:	learn: 0.0045869	total: 5.76s	remaining: 2.02s
518:	learn: 0.0045703	total: 5.77s	remaining: 2.01s
519:	learn: 0.00

672:	learn: 0.0035852	total: 7.44s	remaining: 299ms
673:	learn: 0.0035768	total: 7.46s	remaining: 288ms
674:	learn: 0.0035756	total: 7.48s	remaining: 277ms
675:	learn: 0.0035715	total: 7.49s	remaining: 266ms
676:	learn: 0.0035689	total: 7.5s	remaining: 255ms
677:	learn: 0.0035640	total: 7.51s	remaining: 244ms
678:	learn: 0.0035624	total: 7.52s	remaining: 232ms
679:	learn: 0.0035573	total: 7.53s	remaining: 221ms
680:	learn: 0.0035560	total: 7.54s	remaining: 210ms
681:	learn: 0.0035375	total: 7.55s	remaining: 199ms
682:	learn: 0.0035351	total: 7.55s	remaining: 188ms
683:	learn: 0.0035317	total: 7.57s	remaining: 177ms
684:	learn: 0.0035224	total: 7.58s	remaining: 166ms
685:	learn: 0.0035223	total: 7.58s	remaining: 155ms
686:	learn: 0.0035209	total: 7.59s	remaining: 144ms
687:	learn: 0.0035113	total: 7.61s	remaining: 133ms
688:	learn: 0.0035108	total: 7.62s	remaining: 122ms
689:	learn: 0.0035089	total: 7.62s	remaining: 110ms
690:	learn: 0.0035067	total: 7.63s	remaining: 99.4ms
691:	learn: 

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


(0.015879664275786902, 0.9996029261433949)

In [176]:
A_pred = model_A.predict_proba(x_test_new_A)
B_pred = model_B.predict_proba(x_test_new_B)
C_pred = model_C.predict_proba(x_test_new_C)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [177]:
A_sub = make_country_sub(A_pred, test_A, 'A')
B_sub = make_country_sub(B_pred, test_B, 'B')
C_sub = make_country_sub(C_pred, test_C, 'C')

In [178]:
submission = pd.concat([A_sub, B_sub, C_sub])

In [179]:
submission.to_csv('0226.csv')